In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import glob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/Users/hrachkhachatryan/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
df = pd.read_csv('pos_neg.csv')

In [3]:
df.size

8371056

In [4]:
df[df.pos > df.neg].size

AttributeError: 'DataFrame' object has no attribute 'pos'

In [12]:
for index, row in df.iterrows():
    df.set_value(index, 'timestamp', pd.Timestamp(row['timestamp'], unit='s').date())

/Users/hrachkhachatryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [45]:
df

,Unnamed: 0,timestamp,category_id,topic_id,topic_title,message_number,message_author,message_text,quoteheader,pos,neg,neu,compound
0,0,1514379921,57,2653937,16.5K will be the major resistance for BTC ?,1,AICoin_Official,"As you can see, price started a correction imm...",NaN,0.082,0.041,0.876,0.9201
1,1,1514380918,57,2653937,16.5K will be the major resistance for BTC ?,2,nokati,And&nbsp; who is this Kuang Ren with such a wi...,[],0.112,0.033,0.854,0.7476
2,2,1514381025,57,2653937,16.5K will be the major resistance for BTC ?,3,fabiorem,"Yes, theres a lot of resistance at this level,...",[],0.110,0.000,0.890,0.2144
3,3,1514382464,57,2653937,16.5K will be the major resistance for BTC ?,4,heringasem,With the current price fluctuations the price ...,[],0.127,0.058,0.815,0.4939
4,4,1514384271,57,2653937,16.5K will be the major resistance for BTC ?,5,CuDoCuDau,You should buy bitcoin,"['<div class=""quoteheader""><a href=""https://bi...",0.000,0.000,1.000,0.0000
5,5,1514390315,57,2653937,16.5K will be the major resistance for BTC ?,6,BrewMaster,there are currently a couple of things going o...,[],0.169,0.000,0.831,0.8555
6,6,1514393063,57,2653937,16.5K will be the major resistance for BTC ?,7,CuDoCuDau,I think btc will increase $30000 in next year,"['<div class=""quoteheader""><a href=""https://bi...",0.247,0.000,0.753,0.3182
7,7,1514395791,57,2653937,16.5K will be the major resistance for BTC ?,8,Yaunfitda,"In my view, $16K is another barrier that ...","['<div class=""quoteheader""><a href=""https://bi...",0.128,0.113,0.759,0.5106
8,8,1514397927,57,2653937,16.5K will be the major resistance for BTC ?,9,mrcash02,Bitcoin has already hit $19.000 previously. No...,[],0.123,0.029,0.848,0.9031
9,9,1514405189,57,2653937,16.5K will be the major resistance for BTC ?,10,Gaaara,Nope I guess that can only happen after t...,"['<div class=""quoteheader""><a href=""https://bi...",0.090,0.020,0.889,0.5859


In [13]:
date_topic_groups = df.groupby([df.topic_id, df.timestamp]).groups

In [2]:
clear all output

In [3]:
df.pos[1]

SyntaxError: invalid syntax (<ipython-input-3-d97fb8e095a7>, line 2)

In [14]:
pos_neg_groups = {}

In [15]:
for titleID, timestamp in date_topic_groups.keys():
    pos = 0.
    neg = 0.
    neu = 0.
    j = len(date_topic_groups[(titleID, timestamp)])
    for item in date_topic_groups[(titleID, timestamp)]:
        pos += df.pos[item]
        neg += df.neg[item]
        neu += df.neu[item]
    pos_neg_groups.update({(titleID, timestamp):{'posts':date_topic_groups[(titleID, timestamp)], 'pos': pos/j, 'neg': neg/j, 'neu':neu/j}})

In [16]:
r=0
for i in pos_neg_groups.keys():
    if pos_neg_groups[i]['pos'] > pos_neg_groups[i]['neg'] :
        r+=1
print(r)

3862


In [17]:
len(pos_neg_groups)

5008

In [18]:
grouped_topics = df.groupby(df.topic_id).groups

In [19]:
dict_topic = {}
for tit in tqdm(grouped_topics.keys()):
    sentence = (df.message_text[grouped_topics[tit]].values)
    dict_topic.update({tit: ''.join(str(v) for v in sentence) })

100%|██████████| 1191/1191 [00:00<00:00, 1742.19it/s]


In [20]:
list_message = []
for i in dict_topic.keys():
    list_message.append(dict_topic[i])

In [21]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')  #match any word characters until it reaches a non-word character, like a space.

In [22]:
raw = list_message[0].lower()
tokens = tokenizer.tokenize(raw)  

In [23]:
from stop_words import get_stop_words

# create English stop words list
en_stop = get_stop_words('en')

In [24]:
# remove stop words from tokens
stopped_tokens = [i for i in tokens if not i in en_stop]
# print(stopped_tokens)

In [25]:
remove_words = ['will','s' , 'nbsp', 't', 'com', 'http', 'amp', '1xnk8bc', 'href', 'oto', 'www' ]
stopped_tokens = [i for i in stopped_tokens if not i in remove_words]
# print(stopped_tokens)

In [26]:
from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# Create lemmatizer
wnl = WordNetLemmatizer()

In [27]:
from collections import Counter

In [40]:
texts = []

remove_words = ['will','s' ,'bitcoin', 'just', 'get', 'use', 'now', 'solidx' 'people','bitfinex', 'think', 'maybe', 'imageshack', 'pt', 'em', 'img', 'nbsp', 't', 'com', 'http', 'amp', '1xnk8bc', 'href', 'oto', 'www', 'isn', 'etc', 'etf', 'tr', 'td', 'img', 'ath', 'xt', 'xp', 'php', 'img', 'gt', 'pboc', 'th', 'mtgox', 'cny', 'huobi',
 'm', 'import', 'st', 'lt', 'zhou' 'ok', 'color', 'can', "adam", "bitcoin", 'import', 'http', 'li', 'b', 'style' , 'font', 're', 'le', 'gif','span','hr', 'd' , 'jpg', 'png',  'am5om', 'fud', 'mt', 'th' 'hfebupaeo', 'ftdata', 'zbb', 'imgur', 'bite', 'uztgwi', 'podomatic']

map_words = {
    'btc': 'bitcoin',
    'bcc': 'bitcoin',
    'gbtc': 'bitcoin',
    'bitcoinca': 'bitcoin',
    'better': 'good',
    'increase': 'rise',
    'miner': 'mine',
    'winner': 'win'
}

bow_dict = {}
for m in dict_topic.keys():
    raw = dict_topic[m].lower()
    tokens = tokenizer.tokenize(raw)
    
    stopped_tokens = [i for i in tokens if not i in en_stop]
    stopped_tokens = [i for i in stopped_tokens if not i in remove_words]

    #   lemmatize tokents:  
    lemmatized_tokens = [wnl.lemmatize(i, 'a') for i in stopped_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 'n') for i in lemmatized_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 'v') for i in lemmatized_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 'r') for i in lemmatized_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 's') for i in lemmatized_tokens]
    
    #     stemmed_tokens = [p_stemmer.stem(i) for i in lemmatized_tokens]
    
    #   remove words:
    stemmed_tokens = [i for i in lemmatized_tokens if not i in remove_words]
    stemmed_tokens = [i for i in stemmed_tokens if not i in en_stop]
    d = pd.DataFrame({'z': stemmed_tokens})
    stemmed_tokens = d.replace(map_words)['z'].tolist()
    
    #   remove numeric values:
    alpha_only = [''.join(filter(str.isalpha, i))  for i in stemmed_tokens if len(''.join(filter(str.isalpha, i))) >1]
    bow_dict.update({m:alpha_only})
    texts.append(alpha_only)

In [29]:
import gensim

In [30]:
from gensim import corpora, models
import datetime

a = datetime.datetime.now()

dictionary = corpora.Dictionary(texts)
    

In [31]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [25]:
corpus = []

In [26]:
# keep bag of words in order to get docements for each of the topic
for i in bow_dict.keys():
    bow = dictionary.doc2bow(bow_dict[i])
    corpus.append(bow)
    bow_dict.update({i:bow})

In [39]:
a = datetime.datetime.now()
damodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, passes=20)
b = datetime.datetime.now()
c = b - a

KeyboardInterrupt: 

In [316]:
print(c)

0:05:09.802376


In [33]:
from sklearn.externals import joblib

In [317]:

joblib.dump(damodel, 'LDA_model.pkl') 

['LDA_model.pkl']

In [41]:
damodel = joblib.load('LDA_model.pkl') 

In [42]:
# get LDA topics:
for i in damodel.print_topics(num_topics=50, num_words=10):
    print(i)
    print('-----------------')

(0, '0.028*"inherit" + 0.019*"mine" + 0.015*"price" + 0.015*"top" + 0.014*"valign" + 0.008*"people" + 0.006*"difficulty" + 0.005*"go" + 0.005*"make" + 0.005*"rise"')
-----------------
(1, '0.009*"short" + 0.009*"euro" + 0.007*"buy" + 0.006*"go" + 0.005*"good" + 0.004*"one" + 0.004*"dead" + 0.004*"sell" + 0.004*"greece" + 0.004*"imgflip"')
-----------------
(2, '0.006*"people" + 0.006*"win" + 0.006*"one" + 0.005*"good" + 0.005*"know" + 0.005*"watch" + 0.005*"ico" + 0.004*"entry" + 0.004*"like" + 0.004*"say"')
-----------------
(3, '0.009*"dec" + 0.008*"em" + 0.008*"pt" + 0.008*"sum" + 0.008*"bid" + 0.007*"height" + 0.007*"nov" + 0.007*"twimg" + 0.007*"pb" + 0.006*"medium"')
-----------------
(4, '0.005*"train" + 0.004*"node" + 0.004*"protrade" + 0.004*"thread" + 0.004*"topic" + 0.003*"speed" + 0.003*"profile" + 0.003*"ote" + 0.003*"bullbearanalytics" + 0.002*"org"')
-----------------
(5, '0.007*"like" + 0.006*"time" + 0.006*"see" + 0.005*"market" + 0.005*"say" + 0.004*"make" + 0.004*"ch

In [36]:
final_date_topic_dict = {}

In [43]:
p = 0
n = 0
for titleID, timestamp in pos_neg_groups:
    topics = damodel.get_document_topics(bow_dict[titleID])
    max_prob = 0
    max_t = 0
    for t, prob in topics:
        if prob > max_prob:
            max_prob = prob
            max_t = t
    pos = pos_neg_groups[(titleID, timestamp)]['pos']
    neg = pos_neg_groups[(titleID, timestamp)]['neg']
    neu = pos_neg_groups[(titleID, timestamp)]['neu']
#     print('pos: ', pos, 'neg: ', neg)
    label = 'neu'
    if pos < neg:
        lebel = 'neg'
        final_date_topic_dict.update({(titleID, timestamp):{'maxTopicID': max_t, 'label': 'neg', 'topicProb': max_prob, 'allTopicsWithProb': topics}})
    elif pos > neg:
        label = 'pos'
        final_date_topic_dict.update({(titleID, timestamp):{'maxTopicID': max_t, 'label': 'pos', 'topicProb': max_prob, 'allTopicsWithProb': topics}})
    else:
        final_date_topic_dict.update({(titleID, timestamp):{'maxTopicID': max_t, 'label': 'neu', 'topicProb': max_prob, 'allTopicsWithProb': topics}})

print('pos: ', p)
print('neg: ', n)

ValueError: invalid literal for int() with base 10: 'o'

In [401]:
final_date_topic_dict

{(27683,
  Timestamp('2011-07-10 00:00:00')): {'allTopicsWithProb': [(7, 0.20649093),
   (9, 0.2484943),
   (11, 0.11580125),
   (12,
    0.42744216)], 'label': 'pos', 'maxTopicID': 12, 'topicProb': 0.42744216},
 (27683,
  Timestamp('2011-07-11 00:00:00')): {'allTopicsWithProb': [(7, 0.2052259),
   (9, 0.24757774),
   (11, 0.118294045),
   (12, 0.427131)], 'label': 'pos', 'maxTopicID': 12, 'topicProb': 0.427131},
 (27683,
  Timestamp('2011-07-12 00:00:00')): {'allTopicsWithProb': [(7, 0.21175785),
   (9, 0.25220358),
   (11, 0.10547677),
   (12,
    0.42879042)], 'label': 'neg', 'maxTopicID': 12, 'topicProb': 0.42879042},
 (27683,
  Timestamp('2011-07-13 00:00:00')): {'allTopicsWithProb': [(7, 0.2004717),
   (9, 0.24513154),
   (11, 0.12658596),
   (12,
    0.42603946)], 'label': 'pos', 'maxTopicID': 12, 'topicProb': 0.42603946},
 (27683,
  Timestamp('2011-07-18 00:00:00')): {'allTopicsWithProb': [(7, 0.2042557),
   (9, 0.24717426),
   (11, 0.11989303),
   (12,
    0.42690563)], 'label

In [402]:
c = 0
d = 0
for k,v in final_date_topic_dict.keys():
    if final_date_topic_dict[(k,v)]['label'] == 'pos':
        c+=1
    elif  final_date_topic_dict[(k,v)]['label'] == 'neg':
        d+=1
print(c)
print(d)

3862
931


In [403]:
topic_date_label_dict = {}

In [404]:
for docID, timestamp in final_date_topic_dict:
    max_topic = final_date_topic_dict[(docID, timestamp)]['maxTopicID']
    if (max_topic, timestamp) in topic_date_label_dict:
        topic_date_label_dict[(max_topic, timestamp)].append(final_date_topic_dict[(docID, timestamp)]['label'])
    else:
        topic_date_label_dict[(max_topic, timestamp)] = [final_date_topic_dict[(docID, timestamp)]['label']]

In [405]:
topic_date_label_dict

{(12, Timestamp('2011-07-10 00:00:00')): ['pos'],
 (12, Timestamp('2011-07-11 00:00:00')): ['pos'],
 (12, Timestamp('2011-07-12 00:00:00')): ['neg'],
 (12, Timestamp('2011-07-13 00:00:00')): ['pos'],
 (12, Timestamp('2011-07-18 00:00:00')): ['pos'],
 (12, Timestamp('2014-03-15 00:00:00')): ['pos'],
 (12, Timestamp('2014-03-16 00:00:00')): ['pos'],
 (12, Timestamp('2014-03-17 00:00:00')): ['neg'],
 (12, Timestamp('2017-03-24 00:00:00')): ['pos'],
 (12, Timestamp('2017-03-26 00:00:00')): ['pos'],
 (12, Timestamp('2017-03-27 00:00:00')): ['pos'],
 (11, Timestamp('2011-07-16 00:00:00')): ['pos'],
 (9, Timestamp('2011-07-18 00:00:00')): ['pos'],
 (9, Timestamp('2011-08-03 00:00:00')): ['pos', 'pos'],
 (9, Timestamp('2012-11-27 00:00:00')): ['pos', 'pos'],
 (9, Timestamp('2011-07-23 00:00:00')): ['neg', 'neg'],
 (9, Timestamp('2011-07-24 00:00:00')): ['pos', 'pos', 'pos'],
 (9, Timestamp('2011-07-26 00:00:00')): ['pos', 'pos'],
 (9, Timestamp('2011-08-05 00:00:00')): ['neg', 'pos'],
 (9, Tim

In [406]:
for topicID, timestamp in topic_date_label_dict:
    labels = topic_date_label_dict[(topicID, timestamp)]
    count = len(labels)
    counter = Counter(labels)
    neu = counter['neu']/(0. + count)
    pos = counter['pos']/(0. + count)
    neg = counter['neg']/(0. + count)
    label = 'neu'
    if pos > neg:
        topic_date_label_dict.update({(topicID, timestamp):'pos'})
    elif neg > pos:
        topic_date_label_dict.update({(topicID, timestamp):'neg'})
    else:
        topic_date_label_dict.update({(topicID, timestamp):'neu'})
    

In [407]:
topic_date_label_dict

{(12, Timestamp('2011-07-10 00:00:00')): 'pos',
 (12, Timestamp('2011-07-11 00:00:00')): 'pos',
 (12, Timestamp('2011-07-12 00:00:00')): 'neg',
 (12, Timestamp('2011-07-13 00:00:00')): 'pos',
 (12, Timestamp('2011-07-18 00:00:00')): 'pos',
 (12, Timestamp('2014-03-15 00:00:00')): 'pos',
 (12, Timestamp('2014-03-16 00:00:00')): 'pos',
 (12, Timestamp('2014-03-17 00:00:00')): 'neg',
 (12, Timestamp('2017-03-24 00:00:00')): 'pos',
 (12, Timestamp('2017-03-26 00:00:00')): 'pos',
 (12, Timestamp('2017-03-27 00:00:00')): 'pos',
 (11, Timestamp('2011-07-16 00:00:00')): 'pos',
 (9, Timestamp('2011-07-18 00:00:00')): 'pos',
 (9, Timestamp('2011-08-03 00:00:00')): 'pos',
 (9, Timestamp('2012-11-27 00:00:00')): 'pos',
 (9, Timestamp('2011-07-23 00:00:00')): 'neg',
 (9, Timestamp('2011-07-24 00:00:00')): 'pos',
 (9, Timestamp('2011-07-26 00:00:00')): 'pos',
 (9, Timestamp('2011-08-05 00:00:00')): 'neu',
 (9, Timestamp('2011-08-06 00:00:00')): 'pos',
 (9, Timestamp('2011-08-07 00:00:00')): 'pos',
 

In [410]:
i = 0
for label in topic_date_label_dict.values():
    if(label == 'neu'):
        i+=1
print(i)

291


In [9]:
doc_date_count_dict = {}

In [10]:
# group topic daily counts
for titleID, timestamp in date_topic_groups.keys():
    j = len(date_topic_groups[(titleID, timestamp)])
    
    doc_date_count_dict.update({(titleID, timestamp):{'posts':date_topic_groups[(titleID, timestamp)], 'count': j}})

In [32]:
for titleID, timestamp in doc_date_count_dict:
    topics = damodel.get_document_topics(bow_dict[titleID])
    max_prob = 0
    max_t = 0
    for t, prob in topics:
        if prob > max_prob:
            max_prob = prob
            max_t = t
    
       
    doc_date_count_dict.update({(titleID, timestamp):{'maxTopicID': max_t, 'count':doc_date_count_dict[(titleID, timestamp)]['count'], 'allTopicsWithProb': topics}})

In [33]:
topic_date_count_dict = {}

In [34]:
for docID, timestamp in doc_date_count_dict:
    max_topic = doc_date_count_dict[(docID, timestamp)]['maxTopicID']
    if (max_topic, timestamp) in topic_date_count_dict:
        topic_date_count_dict[(max_topic, timestamp)]+=doc_date_count_dict[(docID, timestamp)]['count']
    else:
        topic_date_count_dict[(max_topic, timestamp)] = doc_date_count_dict[(docID, timestamp)]['count']

In [35]:
topic_date_count_dict

{(12, Timestamp('2011-07-10 00:00:00')): 2,
 (12, Timestamp('2011-07-11 00:00:00')): 1,
 (12, Timestamp('2011-07-12 00:00:00')): 2,
 (12, Timestamp('2011-07-13 00:00:00')): 3,
 (12, Timestamp('2011-07-18 00:00:00')): 2,
 (12, Timestamp('2014-03-15 00:00:00')): 5,
 (12, Timestamp('2014-03-16 00:00:00')): 2,
 (12, Timestamp('2014-03-17 00:00:00')): 2,
 (12, Timestamp('2017-03-24 00:00:00')): 7,
 (12, Timestamp('2017-03-26 00:00:00')): 7,
 (12, Timestamp('2017-03-27 00:00:00')): 2,
 (11, Timestamp('2011-07-16 00:00:00')): 1,
 (9, Timestamp('2011-07-18 00:00:00')): 15,
 (9, Timestamp('2011-08-03 00:00:00')): 20,
 (9, Timestamp('2012-11-27 00:00:00')): 5,
 (9, Timestamp('2011-07-23 00:00:00')): 29,
 (9, Timestamp('2011-07-24 00:00:00')): 16,
 (9, Timestamp('2011-07-26 00:00:00')): 5,
 (9, Timestamp('2011-08-05 00:00:00')): 16,
 (9, Timestamp('2011-08-06 00:00:00')): 4,
 (9, Timestamp('2011-08-07 00:00:00')): 5,
 (9, Timestamp('2011-08-10 00:00:00')): 6,
 (9, Timestamp('2011-08-12 00:00:00')